In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
from photutils import CircularAperture
from photutils import aperture_photometry
from astropy import wcs
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import requests
import time
from bs4 import BeautifulSoup

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
# diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
# medianDir = '/media/mj1e16/PP AV-TV/medians/'
# imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'

In [2]:
diffDir = '/data/mj1e16/kepler/properDiff/'
medianDir = '/data/mj1e16/kepler/medians/'
imageDir = '/data/mj1e16/kepler/cal/'

In [3]:
diffImages = os.listdir(diffDir)
diffImages = [x for x in diffImages if '.fits' in x]

medianImages = os.listdir(medianDir)
medianImages = [x for x in medianImages if '.fits' in x]

images = os.listdir(imageDir)
images = [x for x in images if '.fits' in x]

In [4]:
#unmatchedObjects = ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')
#unmatchedObjects =  ascii.read('/home/mj1e16/keplerPhotometry/sexdiffOutNew2.csv')

In [5]:
firstunmatchedObjects = ascii.read('/home/mj1e16/outTablesImproved/final/firsthalf.csv')

In [6]:
secondunmatchedObjects = ascii.read('/home/mj1e16/outTablesImproved/final/secondhalf.csv')

In [7]:
unmatchedObjects = vstack([firstunmatchedObjects,secondunmatchedObjects])

In [8]:
indexlist = np.linspace(0,len(unmatchedObjects)-1,len(unmatchedObjects))
intexlist = [int(x) for x in indexlist]
print(len(intexlist))
print(len(unmatchedObjects))
print(int(intexlist[-1]))

2614592
2614592
2614591


In [9]:
xpos = np.array(unmatchedObjects['X_IMAGE'])
ypos = np.array(unmatchedObjects['Y_IMAGE'])
imname = np.array(unmatchedObjects['IMAGE_NAME'])
unmatchedObjects['index'] = intexlist
#indexlist = np.linspace(0,len(unmatchedObjects),len(unmatchedObjects)+1)
allnames = np.unique(imname)

In [10]:
uni = np.unique(imname)
print(len(uni))

4296


In [11]:
def load_wcs_from_file(filename,pixCoord,ext=0):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)
    print(filename)
    # Parse the WCS keywords in the primary HDU
    print(int(ext))
    w = wcs.WCS(hdulist[int(ext)].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [12]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='5'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [13]:
def appendToDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='5'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = ''
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'a') as f:
            f.write(bigString)

In [14]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [15]:
def findMatchesNoCondition(image,diffname,tab,match,max_sep = 2.5 * u.arcsec,magsurplus=-3,condition='mag',ratioThresh=10):
    
    df = tab.to_pandas()
    bigdf = match.to_pandas()
    
    number1 = tab['NUMBER'] # difference image
    ra1 = tab['RA']
    dec1 = tab['DEC'] 
    
    number2 = match['NUMBER'] # original image
    ra2 = match['RA']
    dec2 = match['DEC']
    
    #makeDS9RegFile([tab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='red')

    cat1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)
    sep_constraint = d2d < max_sep
    
    objectsFound = number1[sep_constraint] # index of objects found in difference image
    objectsFoundList  = objectsFound.tolist()
    
    matches = idx[sep_constraint] # index of objects found in original,  image
#     matchdf = bigdf[bigdf['NUMBER'].isin(matches)]
    
#     print(objectsFound.tolist(), len(objectsFound.tolist()))
#     print(matches, len(matches))
#     newra1 = matchdf['RA'].tolist()
#     newdec1 = matchdf['DEC'].tolist()

#     newra1 = [match['RA'][x] for x in matches]
#     otherra1 = ra1[sep_constraint].tolist()
    
#     newdec1 = [match['DEC'][x] for x in matches]
    
    #ognumber = matchdf['NUMBER'].tolist()
    ogflux = [match['FLUX_BEST'][x] for x in matches]
    ogMag =  [match['MAG_BEST'][x] for x in matches]
    #ogflux = matchdf['FLUX_BEST'].tolist()
    #matchTab = Table.from_pandas(matchdf)
    
    #print(matchTab)
    
    origdf = df[df['NUMBER'].isin(objectsFound)]
    orignum = origdf['NUMBER'].tolist()
    origflux = [tab['FLUX_BEST'][x] for x in objectsFoundList]
#     numtest = [tab['NUMBER'][x] for x in objectsFoundList]
    #print([[x,y] for x,y in zip(objectsFoundList,numtest)])
    #origflux = origdf['FLUX_BEST'].tolist()
    #print(origdf['NUMBER'].tolist())

#     newra2 = origdf['RA'].tolist()
#     newdec2 = origdf['DEC'].tolist()
#     print([[x,y,z] for x,y,z in zip(newra2,newra1,otherra1)])
#     print([[x,y] for x,y in zip(newra1,newra2)])
#     print([[x,y] for x,y in zip(newdec1,newdec2)])
    
    #print(origdf)
    #appendToDS9RegFile([matchTab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='blue',radius='10')
    
    gain = 110
    fluxdiff = [(x-y) for x,y in zip(ogflux,origflux)]
    #medianFlux = [(x-y) for x,y in zip(ogflux,origflux)]
    magdiff = [-2.5*np.log10(x/y) for x,y in zip(ogflux,fluxdiff)]
    #SNratio = [(x/110)/(((x/((13**0.5)*110))+(y/110))**0.5) for x,y in zip(fluxdiff,origflux)] #+(90**2)
    noise = [((x/gain)+(y/(gain*13)))**0.5 for x,y in zip(ogflux,fluxdiff)]
    SNratio = [(x/gain)/y for x,y in zip(origflux,noise)]
    #SNratio = [(y/gain)/(((z/gain)+(x/((13)*gain)))**0.5) for x,y,z in zip(fluxdiff,origflux,ogflux)]
    #]print(SNratio)
    #print(SNratio)
    #print(magdiff)
    #print([[x,w,y,z] for w,x,y,z in zip(origflux,ogflux,magdiff,fluxdiff)])
    #print(magdiff)
    if condition == 'mag':
        goodNums = [x for x,y in zip(orignum,magdiff) if y < magsurplus] # fine for now, in future change to not rely on the 
        magdiffColumn = [x for x in magdiff if x < magsurplus]            # Y_iMAGE position of the objects
        magorigColumn = [x for x,y in zip(ogMag,magdiff) if y < magsurplus]
        SNratioColumn = [x for x,y in zip(SNratio,magdiff) if y < magsurplus]
    elif condition == 's/n':
        goodNums = [x for x,y in zip(orignum,SNratio) if y > ratioThresh] # fine for now, in future change to not rely on the 
        magdiffColumn = [x for x,y in zip(magdiff,SNratio) if y > ratioThresh]            # Y_iMAGE position of the objects
        magorigColumn = [x for x,y in zip(ogMag,SNratio) if y > ratioThresh]
        SNratioColumn = [x for x in SNratio if x > ratioThresh]
    finaldf = origdf[origdf['NUMBER'].isin(goodNums)]
    #finaldf = matchdf[matchdf['NUMBER'].isin(goodNums)]
    finaldf['magDiff'] = magdiffColumn
    finaldf['MAG_BEST_ORIG'] = magorigColumn
    finaldf['S/N'] = SNratioColumn
    if len(finaldf) != 0:
        #print(finaldf)
        #print('yabba dabba do')
        finalTab = Table.from_pandas(finaldf)

    return finaldf

In [16]:
# image2diff = {}
# image2median = {}
# for x in diffImages:
#     try:
#         results = findName(x)
#         image2diff[results[0]] = x
#         image2median[results[0]] = results[1]
#     except:
#         print(x)
    
# diff = [image2diff[x] for x in imname]
# median = [image2median[x] for x in imname]

# unmatchedObjects['diffImage'] = diff
# unmatchedObjects['medianImage'] = median

In [17]:
def showImage(images,regfile):
    commands = ['ds9']
    for x in range(len(images)):
        commands.extend([images[x],'-regions',regfile,'-scale','mode','zscale'])
    commands.extend(['-frame','lock','image'])
    #print(commands)
    subprocess.call(commands)

In [18]:
def openFromTable(table,ind,imDir=imageDir,diffDir=diffDir,medianDir=medianDir):
    ra = table['RA'][ind]
    dec = table['DEC'][ind]
    xim = table['X_IMAGE'][ind]
    yim = table['Y_IMAGE'][ind]
    image = table['IMAGE_NAME'][ind]
    median = table['MEDIAN_NAME'][ind]
    diff = table['DIFF_NAME'][ind]
    with open('/home/mj1e16/outTablesImproved/testregImage_{}.reg'.format(ind),'w') as f:
        f.write('global color=red\nimage\ncircle({0},{1},{2})\ncircle({0},{1},{3})'.format(xim,yim,5.,60.))
    with open('/home/mj1e16/outTablesImproved/testregWCS_{}.reg'.format(ind),'w') as f:
        f.write('global color=red\nICRS\ncircle({0},{1},{2})\ncircle({0},{1},{3})'.format(ra,dec,3./3600,24./3600.))
    
    command = ['ds9']
    images= [imDir+image,diffDir+diff,medianDir+median]
    for x in range(3):
        command.extend([images[x],'-regions','/home/mj1e16/outTablesImproved/testregImage_{}.reg'.format(ind),'-scale','mode','zscale'])
    getPanStarCutOut(ra,dec,size=240)
    panCommand = ['/home/mj1e16/cutOuts/cutout.fits','-regions','/home/mj1e16/outTablesImproved/testregWCS_{}.reg'.format(ind)]
    command.extend(panCommand)
    command.extend(['-frame','lock','wcs'])
    print(ra,dec,image)
    subprocess.call(command)

In [19]:
def getPanStarCutOut(ra,dec,size=240,outname='cutout.fits'):
    url = 'https://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={}%2C{}&filter=color&filter=g&filter=r&filter=i&filter=z&filter=y&filetypes=stack&auxiliary=data&size={}&output_size=0&verbose=0&autoscale=99.500000&catlist='.format(ra,dec,size)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    th = soup.findAll('td')
    try:
        link = th[1]
        downLink = link.findAll('img')
        downCut = downLink[0]['src']
        loc1 = downCut.find('cgi-bin')
        loc2 = downCut.find('=/') + len('=/')
        if loc1 != -1:
            if loc2 != -1:
                newCut = downCut[:loc1] + downCut[loc2:]

        loc3 = newCut.find('&')
        smallCut = newCut[:loc3]
        http = 'https:'+smallCut
        print(http)
        subprocess.call(['wget','-O','/home/mj1e16/cutOuts/{}'.format(outname),http])
    except:
        print('WARNING - panstarrs image note found ra = {}, dec = {}'.format(ra,dec))


In [20]:
def findOtherOccurences(matchtab,unmatchtab):
    
    ra = matchtab['RA'].tolist()
    dec = matchtab['DEC'].tolist()
    imageName = matchtab['IMAGE_NAME'].tolist()
    #df = matchtab.to_pandas()
    df = unmatchtab.to_pandas()
    maxsep = 2./3600.
    intwo = []
    for x in range(len(ra)):
        df2 = df[(df['RA']>=ra[x]-maxsep)&(df['RA']<=ra[x]+maxsep)&(df['DEC']>=dec[x]-maxsep)&(df['DEC']<=dec[x]+maxsep)&(df['IMAGE_NAME']!=imageName[x])]
        if len(df2) > 0:
            print('Object Match ',x)
            if len(intwo) == 0:
                intwo = Table.from_pandas(df2)
            else:
                intwostep = Table.from_pandas(df2)
                intwo = vstack([intwo,intwostep])
    return intwo

In [59]:
matchInTwo = findOtherOccurences(match25mag,unmatchedObjects)

('Object Match ', 96)
('Object Match ', 117)
('Object Match ', 136)
('Object Match ', 157)
('Object Match ', 199)
('Object Match ', 240)
('Object Match ', 311)
('Object Match ', 326)
('Object Match ', 350)
('Object Match ', 366)
('Object Match ', 450)
('Object Match ', 524)
('Object Match ', 552)
('Object Match ', 575)
('Object Match ', 595)
('Object Match ', 614)
('Object Match ', 619)
('Object Match ', 704)
('Object Match ', 707)
('Object Match ', 729)
('Object Match ', 732)
('Object Match ', 743)
('Object Match ', 749)
('Object Match ', 760)
('Object Match ', 769)
('Object Match ', 770)
('Object Match ', 779)
('Object Match ', 794)
('Object Match ', 837)


In [61]:
print(len(matchInTwo))

438


In [60]:
matchInTwo

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index
int64,float64,float64,float64,float64,float64,float64,float64,float64,str34,str18,str32,int64
58,17503.52,35.19302,14.3922,932.3539,116.5391,0.006,292.23517720674187,43.093748559971694,kplr2010078174524_ffi-cal.fits[62],meidian_62_62.fits,diff_imNo31_channel62_ccd62.fits,3847
62,8324.137,36.30497,15.1992,928.2188,115.317,0.001,292.2352425738186,43.093829038319626,kplr2009351005245_ffi-cal.fits[38],meidian_62_38.fits,diff_imNo28_channel62_ccd38.fits,45144
272,14464.79,20.92687,14.5992,933.5084,121.5713,0.013,292.2354081827335,43.094110112300484,kplr2009115053616_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo48_channel62_ccd46.fits,67729
254,27890.84,19.70852,13.8863,933.8064,121.8305,0.003,292.23523228459015,43.09369324562835,kplr2009115131122_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo49_channel62_ccd46.fits,68000
284,14918.87,20.79213,14.5657,933.5297,121.597,0.028,292.2354013394543,43.09407358605149,kplr2009114204835_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo15_channel62_ccd46.fits,79187
93,1167.716,19.66327,17.3317,928.0181,120.0141,0.005,292.2353616272357,43.09375628721139,kplr2011208112727_ffi-cal.fits[22],meidian_62_22.fits,diff_imNo29_channel62_ccd22.fits,100652
99,28529.88,42.24063,13.8617,928.2314,115.4032,0.006,292.23520210765673,43.093815199769956,kplr2012310200152_ffi-cal.fits[38],meidian_62_38.fits,diff_imNo23_channel62_ccd38.fits,105933
95,37050.28,33.54249,13.578,932.3477,116.5112,0.011,292.23520626446077,43.09377091210652,kplr2013065115251_ffi-cal.fits[62],meidian_62_62.fits,diff_imNo47_channel62_ccd62.fits,123919
258,28596.88,19.7365,13.8592,933.8041,121.8328,0.004,292.23523707706016,43.09369242823554,kplr2009115173611_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo43_channel62_ccd46.fits,178321


In [21]:
matchesSN = []
for x in range(len(uni)):
    name = uni[x]
    print(name,x)
    diffname = unmatchedObjects['DIFF_NAME'][x]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['IMAGE_NAME'] == name]
    minitab = Table.from_pandas(mindf)
    indexing = range(len(minitab))
    minitab['NUMBER'] = indexing
                
    try:
        matchtab = ascii.read('/home/mj1e16/outTablesImproved/imagecsvredo/'+name+'.csv')
        indexing = range(len(matchtab))
        matchtab['NUMBER'] = indexing
        
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()

    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,max_sep=2.5*u.arcsec,condition='mag',magsurplus=-1)
    onematchTab = Table.from_pandas(onematch)
    if len(onematch) != 0:
        if len(matchesSN) == 0:
            onematchTab = Table.from_pandas(onematch)
            matchesSN = onematchTab

        else: 
            #onematchTab = Table.from_pandas(onematch)
            #print(onematchTab)
            for x in range(len(onematchTab)):
                matchesSN.add_row(onematchTab[x])
                #print(onematchTab)
                #matches  = vstack(matches,onematchTab)
                #print(matches)

('kplr2009114174833_ffi-cal.fits[10]', 0)
('kplr2009114174833_ffi-cal.fits[11]', 1)
('kplr2009114174833_ffi-cal.fits[12]', 2)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

('kplr2009114174833_ffi-cal.fits[13]', 3)
('kplr2009114174833_ffi-cal.fits[14]', 4)
('kplr2009114174833_ffi-cal.fits[15]', 5)
('kplr2009114174833_ffi-cal.fits[16]', 6)
('kplr2009114174833_ffi-cal.fits[17]', 7)
('kplr2009114174833_ffi-cal.fits[18]', 8)
('kplr2009114174833_ffi-cal.fits[19]', 9)
('kplr2009114174833_ffi-cal.fits[1]', 10)
('kplr2009114174833_ffi-cal.fits[20]', 11)
('kplr2009114174833_ffi-cal.fits[21]', 12)
('kplr2009114174833_ffi-cal.fits[22]', 13)
('kplr2009114174833_ffi-cal.fits[23]', 14)
('kplr2009114174833_ffi-cal.fits[24]', 15)
('kplr2009114174833_ffi-cal.fits[25]', 16)
('kplr2009114174833_ffi-cal.fits[26]', 17)
('kplr2009114174833_ffi-cal.fits[27]', 18)
('kplr2009114174833_ffi-cal.fits[28]', 19)
('kplr2009114174833_ffi-cal.fits[29]', 20)
('kplr2009114174833_ffi-cal.fits[2]', 21)
('kplr2009114174833_ffi-cal.fits[30]', 22)
('kplr2009114174833_ffi-cal.fits[31]', 23)
('kplr2009114174833_ffi-cal.fits[32]', 24)
('kplr2009114174833_ffi-cal.fits[33]', 25)
('kplr2009114174833_

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in log10


('kplr2009114174833_ffi-cal.fits[56]', 50)
('kplr2009114174833_ffi-cal.fits[57]', 51)
('kplr2009114174833_ffi-cal.fits[58]', 52)
('kplr2009114174833_ffi-cal.fits[59]', 53)
('kplr2009114174833_ffi-cal.fits[5]', 54)
('kplr2009114174833_ffi-cal.fits[60]', 55)
('kplr2009114174833_ffi-cal.fits[61]', 56)
('kplr2009114174833_ffi-cal.fits[62]', 57)
('kplr2009114174833_ffi-cal.fits[63]', 58)
('kplr2009114174833_ffi-cal.fits[64]', 59)
('kplr2009114174833_ffi-cal.fits[65]', 60)
('kplr2009114174833_ffi-cal.fits[66]', 61)
('kplr2009114174833_ffi-cal.fits[67]', 62)
('kplr2009114174833_ffi-cal.fits[68]', 63)
('kplr2009114174833_ffi-cal.fits[69]', 64)
('kplr2009114174833_ffi-cal.fits[6]', 65)
('kplr2009114174833_ffi-cal.fits[70]', 66)
('kplr2009114174833_ffi-cal.fits[71]', 67)
('kplr2009114174833_ffi-cal.fits[72]', 68)
('kplr2009114174833_ffi-cal.fits[73]', 69)
('kplr2009114174833_ffi-cal.fits[74]', 70)
('kplr2009114174833_ffi-cal.fits[75]', 71)
('kplr2009114174833_ffi-cal.fits[76]', 72)
('kplr2009114

...

('kplr2012179140901_ffi-cal.fits[58]', 3548)
('kplr2012179140901_ffi-cal.fits[59]', 3549)
('kplr2012179140901_ffi-cal.fits[60]', 3550)
('kplr2012179140901_ffi-cal.fits[61]', 3551)
('kplr2012179140901_ffi-cal.fits[62]', 3552)
('kplr2012179140901_ffi-cal.fits[63]', 3553)
('kplr2012179140901_ffi-cal.fits[64]', 3554)
('kplr2012179140901_ffi-cal.fits[65]', 3555)
('kplr2012179140901_ffi-cal.fits[66]', 3556)
('kplr2012179140901_ffi-cal.fits[67]', 3557)
('kplr2012179140901_ffi-cal.fits[68]', 3558)
('kplr2012179140901_ffi-cal.fits[69]', 3559)
('kplr2012179140901_ffi-cal.fits[70]', 3560)
('kplr2012179140901_ffi-cal.fits[71]', 3561)
('kplr2012179140901_ffi-cal.fits[72]', 3562)
('kplr2012179140901_ffi-cal.fits[73]', 3563)
('kplr2012179140901_ffi-cal.fits[74]', 3564)
('kplr2012179140901_ffi-cal.fits[75]', 3565)
('kplr2012179140901_ffi-cal.fits[76]', 3566)
('kplr2012179140901_ffi-cal.fits[77]', 3567)
('kplr2012179140901_ffi-cal.fits[78]', 3568)
('kplr2012179140901_ffi-cal.fits[79]', 3569)
('kplr2012

('kplr2012242195726_ffi-cal.fits[81]', 3731)
('kplr2012242195726_ffi-cal.fits[82]', 3732)
('kplr2012242195726_ffi-cal.fits[83]', 3733)
('kplr2012242195726_ffi-cal.fits[84]', 3734)
('kplr2012242195726_ffi-cal.fits[9]', 3735)
('kplr2012277203051_ffi-cal.fits[10]', 3736)
('kplr2012277203051_ffi-cal.fits[11]', 3737)
('kplr2012277203051_ffi-cal.fits[12]', 3738)
('kplr2012277203051_ffi-cal.fits[13]', 3739)
('kplr2012277203051_ffi-cal.fits[14]', 3740)
('kplr2012277203051_ffi-cal.fits[15]', 3741)
('kplr2012277203051_ffi-cal.fits[16]', 3742)
('kplr2012277203051_ffi-cal.fits[17]', 3743)
('kplr2012277203051_ffi-cal.fits[18]', 3744)
('kplr2012277203051_ffi-cal.fits[19]', 3745)
('kplr2012277203051_ffi-cal.fits[1]', 3746)
('kplr2012277203051_ffi-cal.fits[20]', 3747)
('kplr2012277203051_ffi-cal.fits[21]', 3748)
('kplr2012277203051_ffi-cal.fits[22]', 3749)
('kplr2012277203051_ffi-cal.fits[23]', 3750)
('kplr2012277203051_ffi-cal.fits[24]', 3751)
('kplr2012277203051_ffi-cal.fits[25]', 3752)
('kplr201227

('kplr2012341215621_ffi-cal.fits[27]', 3914)
('kplr2012341215621_ffi-cal.fits[28]', 3915)
('kplr2012341215621_ffi-cal.fits[29]', 3916)
('kplr2012341215621_ffi-cal.fits[2]', 3917)
('kplr2012341215621_ffi-cal.fits[30]', 3918)
('kplr2012341215621_ffi-cal.fits[31]', 3919)
('kplr2012341215621_ffi-cal.fits[32]', 3920)
('kplr2012341215621_ffi-cal.fits[33]', 3921)
('kplr2012341215621_ffi-cal.fits[34]', 3922)
('kplr2012341215621_ffi-cal.fits[35]', 3923)
('kplr2012341215621_ffi-cal.fits[36]', 3924)
('kplr2012341215621_ffi-cal.fits[37]', 3925)
('kplr2012341215621_ffi-cal.fits[38]', 3926)
('kplr2012341215621_ffi-cal.fits[39]', 3927)
('kplr2012341215621_ffi-cal.fits[3]', 3928)
('kplr2012341215621_ffi-cal.fits[40]', 3929)
('kplr2012341215621_ffi-cal.fits[41]', 3930)
('kplr2012341215621_ffi-cal.fits[42]', 3931)
('kplr2012341215621_ffi-cal.fits[43]', 3932)
('kplr2012341215621_ffi-cal.fits[44]', 3933)
('kplr2012341215621_ffi-cal.fits[45]', 3934)
('kplr2012341215621_ffi-cal.fits[46]', 3935)
('kplr201234

('kplr2013038133130_ffi-cal.fits[48]', 4097)
('kplr2013038133130_ffi-cal.fits[49]', 4098)
('kplr2013038133130_ffi-cal.fits[4]', 4099)
('kplr2013038133130_ffi-cal.fits[50]', 4100)
('kplr2013038133130_ffi-cal.fits[51]', 4101)
('kplr2013038133130_ffi-cal.fits[52]', 4102)
('kplr2013038133130_ffi-cal.fits[53]', 4103)
('kplr2013038133130_ffi-cal.fits[54]', 4104)
('kplr2013038133130_ffi-cal.fits[55]', 4105)
('kplr2013038133130_ffi-cal.fits[56]', 4106)
('kplr2013038133130_ffi-cal.fits[57]', 4107)
('kplr2013038133130_ffi-cal.fits[58]', 4108)
('kplr2013038133130_ffi-cal.fits[59]', 4109)
('kplr2013038133130_ffi-cal.fits[60]', 4110)
('kplr2013038133130_ffi-cal.fits[61]', 4111)
('kplr2013038133130_ffi-cal.fits[62]', 4112)
('kplr2013038133130_ffi-cal.fits[63]', 4113)
('kplr2013038133130_ffi-cal.fits[64]', 4114)
('kplr2013038133130_ffi-cal.fits[65]', 4115)
('kplr2013038133130_ffi-cal.fits[66]', 4116)
('kplr2013038133130_ffi-cal.fits[67]', 4117)
('kplr2013038133130_ffi-cal.fits[68]', 4118)
('kplr20130

('kplr2013098115308_ffi-cal.fits[70]', 4280)
('kplr2013098115308_ffi-cal.fits[71]', 4281)
('kplr2013098115308_ffi-cal.fits[72]', 4282)
('kplr2013098115308_ffi-cal.fits[73]', 4283)
('kplr2013098115308_ffi-cal.fits[74]', 4284)
('kplr2013098115308_ffi-cal.fits[75]', 4285)
('kplr2013098115308_ffi-cal.fits[76]', 4286)
('kplr2013098115308_ffi-cal.fits[77]', 4287)
('kplr2013098115308_ffi-cal.fits[78]', 4288)
('kplr2013098115308_ffi-cal.fits[79]', 4289)
('kplr2013098115308_ffi-cal.fits[80]', 4290)
('kplr2013098115308_ffi-cal.fits[81]', 4291)
('kplr2013098115308_ffi-cal.fits[82]', 4292)
('kplr2013098115308_ffi-cal.fits[83]', 4293)
('kplr2013098115308_ffi-cal.fits[84]', 4294)
('kplr2013098115308_ffi-cal.fits[9]', 4295)


In [34]:
print(len(matchesSN))

857


In [38]:
matchesSN

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,MAGERR_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index,magDiff,MAG_BEST_ORIG,S/N
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str34,str18,str32,int64,float64,float64,float64
442,3081.252,6.996094,16.2782,0.0025,817.5698,251.1989,0.154,285.3085242660774,38.64320337442705,kplr2009114174833_ffi-cal.fits[12],meidian_72_12.fits,diff_imNo11_channel72_ccd12.fits,809318,-2.051571026739004,16.1003,0.36454307847359574
345,1563.474,5.890203,17.0148,0.0041,977.2311,335.0007,0.155,283.4488845006899,46.91229413625908,kplr2009114174833_ffi-cal.fits[13],meidian_9_13.fits,diff_imNo11_channel9_ccd13.fits,1708991,-4.065613299233601,16.9888,0.16132052040516148
328,1447.371,5.992507,17.0986,0.0045,881.0323,160.3248,0.282,281.2626986253877,48.092472568384956,kplr2009114174833_ffi-cal.fits[15],meidian_11_15.fits,diff_imNo11_channel11_ccd15.fits,225274,-2.114461736361008,16.9315,0.17006099174451444
382,1633.5,6.138269,16.9672,0.0041,433.0373,179.9004,0.343,282.4562432093736,48.71368743412436,kplr2009114174833_ffi-cal.fits[16],meidian_12_16.fits,diff_imNo11_channel12_ccd16.fits,1963927,-1.9795994886337922,16.776,0.1961710921828971
272,2189.427,6.83307,16.6492,0.0034,804.2537,448.7743,0.337,284.5588528253113,44.51322961355488,kplr2009114174833_ffi-cal.fits[19],meidian_27_19.fits,diff_imNo11_channel27_ccd19.fits,1127188,-1.2588115646345834,16.2405,0.32068959205906167
346,2139.691,6.83307,16.6741,0.0035,91.0074,292.5105,0.469,285.02991137598053,43.780980764606234,kplr2009114174833_ffi-cal.fits[19],meidian_27_19.fits,diff_imNo11_channel27_ccd19.fits,1127262,-1.049369785748642,16.1544,0.34707069486162473
0,14608.01,11.79924,14.5885,0.0009,849.4122,32.2863,0.739,288.4724636796787,43.80081740402751,kplr2009114174833_ffi-cal.fits[21],meidian_45_21.fits,diff_imNo11_channel45_ccd21.fits,1827093,-2.1459148315518672,14.4266,1.6680739244494758
328,8640.346,9.385054,15.1587,0.0012,386.4077,513.6817,0.587,287.4616317565798,43.91652339662211,kplr2009114174833_ffi-cal.fits[21],meidian_45_21.fits,diff_imNo11_channel45_ccd21.fits,1827421,-1.3879303146710897,14.8043,1.1881178830055494
406,1877.488,6.015483,16.8161,0.0035,342.213,242.1214,0.17,285.68313915240736,43.04508557705439,kplr2009114174833_ffi-cal.fits[24],meidian_48_24.fits,diff_imNo11_channel48_ccd24.fits,1738427,-2.854576465862211,16.7348,0.20371355675370095


In [39]:
df = matchesSN.to_pandas()
df2 = df[df['magDiff']<= -3.0]
print(len(df2))

85


In [22]:
ascii.write(matchesSN,'/home/mj1e16/outTablesImproved/final/magone.csv',format='csv',fast_writer=False)

In [23]:
matchesSN = []
for x in range(len(uni)):
    name = uni[x]
    print(name,x)
    diffname = unmatchedObjects['DIFF_NAME'][x]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['IMAGE_NAME'] == name]
    minitab = Table.from_pandas(mindf)
    indexing = range(len(minitab))
    minitab['NUMBER'] = indexing
                
    try:
        matchtab = ascii.read('/home/mj1e16/outTablesImproved/imagecsvredo/'+name+'.csv')
        indexing = range(len(matchtab))
        matchtab['NUMBER'] = indexing
        
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()

    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,max_sep=2.5*u.arcsec,condition='s/n',ratioThresh=10)
    onematchTab = Table.from_pandas(onematch)
    if len(onematch) != 0:
        if len(matchesSN) == 0:
            onematchTab = Table.from_pandas(onematch)
            matchesSN = onematchTab

        else: 
            #onematchTab = Table.from_pandas(onematch)
            #print(onematchTab)
            for x in range(len(onematchTab)):
                matchesSN.add_row(onematchTab[x])
                #print(onematchTab)
                #matches  = vstack(matches,onematchTab)
                #print(matches)

('kplr2009114174833_ffi-cal.fits[10]', 0)
('kplr2009114174833_ffi-cal.fits[11]', 1)
('kplr2009114174833_ffi-cal.fits[12]', 2)
('kplr2009114174833_ffi-cal.fits[13]', 3)
('kplr2009114174833_ffi-cal.fits[14]', 4)
('kplr2009114174833_ffi-cal.fits[15]', 5)
('kplr2009114174833_ffi-cal.fits[16]', 6)
('kplr2009114174833_ffi-cal.fits[17]', 7)
('kplr2009114174833_ffi-cal.fits[18]', 8)
('kplr2009114174833_ffi-cal.fits[19]', 9)
('kplr2009114174833_ffi-cal.fits[1]', 10)
('kplr2009114174833_ffi-cal.fits[20]', 11)
('kplr2009114174833_ffi-cal.fits[21]', 12)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

('kplr2009114174833_ffi-cal.fits[22]', 13)
('kplr2009114174833_ffi-cal.fits[23]', 14)
('kplr2009114174833_ffi-cal.fits[24]', 15)
('kplr2009114174833_ffi-cal.fits[25]', 16)
('kplr2009114174833_ffi-cal.fits[26]', 17)
('kplr2009114174833_ffi-cal.fits[27]', 18)
('kplr2009114174833_ffi-cal.fits[28]', 19)
('kplr2009114174833_ffi-cal.fits[29]', 20)
('kplr2009114174833_ffi-cal.fits[2]', 21)
('kplr2009114174833_ffi-cal.fits[30]', 22)
('kplr2009114174833_ffi-cal.fits[31]', 23)
('kplr2009114174833_ffi-cal.fits[32]', 24)
('kplr2009114174833_ffi-cal.fits[33]', 25)
('kplr2009114174833_ffi-cal.fits[34]', 26)
('kplr2009114174833_ffi-cal.fits[35]', 27)
('kplr2009114174833_ffi-cal.fits[36]', 28)
('kplr2009114174833_ffi-cal.fits[37]', 29)
('kplr2009114174833_ffi-cal.fits[38]', 30)
('kplr2009114174833_ffi-cal.fits[39]', 31)
('kplr2009114174833_ffi-cal.fits[3]', 32)
('kplr2009114174833_ffi-cal.fits[40]', 33)
('kplr2009114174833_ffi-cal.fits[41]', 34)
('kplr2009114174833_ffi-cal.fits[42]', 35)
('kplr2009114

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in log10


('kplr2009114174833_ffi-cal.fits[56]', 50)
('kplr2009114174833_ffi-cal.fits[57]', 51)
('kplr2009114174833_ffi-cal.fits[58]', 52)
('kplr2009114174833_ffi-cal.fits[59]', 53)
('kplr2009114174833_ffi-cal.fits[5]', 54)
('kplr2009114174833_ffi-cal.fits[60]', 55)
('kplr2009114174833_ffi-cal.fits[61]', 56)
('kplr2009114174833_ffi-cal.fits[62]', 57)
('kplr2009114174833_ffi-cal.fits[63]', 58)
('kplr2009114174833_ffi-cal.fits[64]', 59)
('kplr2009114174833_ffi-cal.fits[65]', 60)
('kplr2009114174833_ffi-cal.fits[66]', 61)
('kplr2009114174833_ffi-cal.fits[67]', 62)
('kplr2009114174833_ffi-cal.fits[68]', 63)
('kplr2009114174833_ffi-cal.fits[69]', 64)
('kplr2009114174833_ffi-cal.fits[6]', 65)
('kplr2009114174833_ffi-cal.fits[70]', 66)
('kplr2009114174833_ffi-cal.fits[71]', 67)
('kplr2009114174833_ffi-cal.fits[72]', 68)
('kplr2009114174833_ffi-cal.fits[73]', 69)
('kplr2009114174833_ffi-cal.fits[74]', 70)
('kplr2009114174833_ffi-cal.fits[75]', 71)
('kplr2009114174833_ffi-cal.fits[76]', 72)
('kplr2009114

...

('kplr2012179140901_ffi-cal.fits[58]', 3548)
('kplr2012179140901_ffi-cal.fits[59]', 3549)
('kplr2012179140901_ffi-cal.fits[60]', 3550)
('kplr2012179140901_ffi-cal.fits[61]', 3551)
('kplr2012179140901_ffi-cal.fits[62]', 3552)
('kplr2012179140901_ffi-cal.fits[63]', 3553)
('kplr2012179140901_ffi-cal.fits[64]', 3554)
('kplr2012179140901_ffi-cal.fits[65]', 3555)
('kplr2012179140901_ffi-cal.fits[66]', 3556)
('kplr2012179140901_ffi-cal.fits[67]', 3557)
('kplr2012179140901_ffi-cal.fits[68]', 3558)
('kplr2012179140901_ffi-cal.fits[69]', 3559)
('kplr2012179140901_ffi-cal.fits[70]', 3560)
('kplr2012179140901_ffi-cal.fits[71]', 3561)
('kplr2012179140901_ffi-cal.fits[72]', 3562)
('kplr2012179140901_ffi-cal.fits[73]', 3563)
('kplr2012179140901_ffi-cal.fits[74]', 3564)
('kplr2012179140901_ffi-cal.fits[75]', 3565)
('kplr2012179140901_ffi-cal.fits[76]', 3566)
('kplr2012179140901_ffi-cal.fits[77]', 3567)
('kplr2012179140901_ffi-cal.fits[78]', 3568)
('kplr2012179140901_ffi-cal.fits[79]', 3569)
('kplr2012

('kplr2012242195726_ffi-cal.fits[81]', 3731)
('kplr2012242195726_ffi-cal.fits[82]', 3732)
('kplr2012242195726_ffi-cal.fits[83]', 3733)
('kplr2012242195726_ffi-cal.fits[84]', 3734)
('kplr2012242195726_ffi-cal.fits[9]', 3735)
('kplr2012277203051_ffi-cal.fits[10]', 3736)
('kplr2012277203051_ffi-cal.fits[11]', 3737)
('kplr2012277203051_ffi-cal.fits[12]', 3738)
('kplr2012277203051_ffi-cal.fits[13]', 3739)
('kplr2012277203051_ffi-cal.fits[14]', 3740)
('kplr2012277203051_ffi-cal.fits[15]', 3741)
('kplr2012277203051_ffi-cal.fits[16]', 3742)
('kplr2012277203051_ffi-cal.fits[17]', 3743)
('kplr2012277203051_ffi-cal.fits[18]', 3744)
('kplr2012277203051_ffi-cal.fits[19]', 3745)
('kplr2012277203051_ffi-cal.fits[1]', 3746)
('kplr2012277203051_ffi-cal.fits[20]', 3747)
('kplr2012277203051_ffi-cal.fits[21]', 3748)
('kplr2012277203051_ffi-cal.fits[22]', 3749)
('kplr2012277203051_ffi-cal.fits[23]', 3750)
('kplr2012277203051_ffi-cal.fits[24]', 3751)
('kplr2012277203051_ffi-cal.fits[25]', 3752)
('kplr201227

('kplr2012341215621_ffi-cal.fits[27]', 3914)
('kplr2012341215621_ffi-cal.fits[28]', 3915)
('kplr2012341215621_ffi-cal.fits[29]', 3916)
('kplr2012341215621_ffi-cal.fits[2]', 3917)
('kplr2012341215621_ffi-cal.fits[30]', 3918)
('kplr2012341215621_ffi-cal.fits[31]', 3919)
('kplr2012341215621_ffi-cal.fits[32]', 3920)
('kplr2012341215621_ffi-cal.fits[33]', 3921)
('kplr2012341215621_ffi-cal.fits[34]', 3922)
('kplr2012341215621_ffi-cal.fits[35]', 3923)
('kplr2012341215621_ffi-cal.fits[36]', 3924)
('kplr2012341215621_ffi-cal.fits[37]', 3925)
('kplr2012341215621_ffi-cal.fits[38]', 3926)
('kplr2012341215621_ffi-cal.fits[39]', 3927)
('kplr2012341215621_ffi-cal.fits[3]', 3928)
('kplr2012341215621_ffi-cal.fits[40]', 3929)
('kplr2012341215621_ffi-cal.fits[41]', 3930)
('kplr2012341215621_ffi-cal.fits[42]', 3931)
('kplr2012341215621_ffi-cal.fits[43]', 3932)
('kplr2012341215621_ffi-cal.fits[44]', 3933)
('kplr2012341215621_ffi-cal.fits[45]', 3934)
('kplr2012341215621_ffi-cal.fits[46]', 3935)
('kplr201234

('kplr2013038133130_ffi-cal.fits[48]', 4097)
('kplr2013038133130_ffi-cal.fits[49]', 4098)
('kplr2013038133130_ffi-cal.fits[4]', 4099)
('kplr2013038133130_ffi-cal.fits[50]', 4100)
('kplr2013038133130_ffi-cal.fits[51]', 4101)
('kplr2013038133130_ffi-cal.fits[52]', 4102)
('kplr2013038133130_ffi-cal.fits[53]', 4103)
('kplr2013038133130_ffi-cal.fits[54]', 4104)
('kplr2013038133130_ffi-cal.fits[55]', 4105)
('kplr2013038133130_ffi-cal.fits[56]', 4106)
('kplr2013038133130_ffi-cal.fits[57]', 4107)
('kplr2013038133130_ffi-cal.fits[58]', 4108)
('kplr2013038133130_ffi-cal.fits[59]', 4109)
('kplr2013038133130_ffi-cal.fits[60]', 4110)
('kplr2013038133130_ffi-cal.fits[61]', 4111)
('kplr2013038133130_ffi-cal.fits[62]', 4112)
('kplr2013038133130_ffi-cal.fits[63]', 4113)
('kplr2013038133130_ffi-cal.fits[64]', 4114)
('kplr2013038133130_ffi-cal.fits[65]', 4115)
('kplr2013038133130_ffi-cal.fits[66]', 4116)
('kplr2013038133130_ffi-cal.fits[67]', 4117)
('kplr2013038133130_ffi-cal.fits[68]', 4118)
('kplr20130

('kplr2013098115308_ffi-cal.fits[70]', 4280)
('kplr2013098115308_ffi-cal.fits[71]', 4281)
('kplr2013098115308_ffi-cal.fits[72]', 4282)
('kplr2013098115308_ffi-cal.fits[73]', 4283)
('kplr2013098115308_ffi-cal.fits[74]', 4284)
('kplr2013098115308_ffi-cal.fits[75]', 4285)
('kplr2013098115308_ffi-cal.fits[76]', 4286)
('kplr2013098115308_ffi-cal.fits[77]', 4287)
('kplr2013098115308_ffi-cal.fits[78]', 4288)
('kplr2013098115308_ffi-cal.fits[79]', 4289)
('kplr2013098115308_ffi-cal.fits[80]', 4290)
('kplr2013098115308_ffi-cal.fits[81]', 4291)
('kplr2013098115308_ffi-cal.fits[82]', 4292)
('kplr2013098115308_ffi-cal.fits[83]', 4293)
('kplr2013098115308_ffi-cal.fits[84]', 4294)
('kplr2013098115308_ffi-cal.fits[9]', 4295)


In [69]:
matchesSN

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,MAGERR_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index,magDiff,MAG_BEST_ORIG,S/N
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str34,str18,str32,int64,float64,float64,float64
0,14608.01,11.79924,14.5885,0.0009,849.4122,32.2863,0.739,288.4724636796787,43.80081740402751,kplr2009114174833_ffi-cal.fits[21],meidian_45_21.fits,diff_imNo11_channel45_ccd21.fits,1827093,-2.1459148315518672,14.4266,10.639220778058874
696,28890.03,7.8394,13.8481,0.0003,933.8094,121.833,0.028,292.2352304264401,43.0936891516194,kplr2009115080620_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo17_channel62_ccd46.fits,1289012,-1.0900206852560455,13.3526,12.721511218627567
705,27857.69,7.717424,13.8876,0.0003,933.8091,121.8326,0.028,292.23523049244886,43.0936897017523,kplr2009115131122_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo49_channel62_ccd46.fits,252204,-1.0698737322776786,13.3803,12.421120385934845
753,28553.71,7.754268,13.8608,0.0003,933.8099,121.8334,0.028,292.2352301034367,43.09368848492147,kplr2009115173611_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo43_channel62_ccd46.fits,656885,-1.085702096380591,13.3628,12.631971741665541
657,25872.45,7.842578,13.9679,0.0003,933.8087,121.8302,0.029,292.23522908953754,43.09369218659433,kplr2009116035924_ffi-cal.fits[46],meidian_62_46.fits,diff_imNo38_channel62_ccd46.fits,1809590,-1.0131784498391505,13.4253,11.768896060080975
540,162063.1,585.291,11.9758,0.0039,800.6366,336.6361,0.134,289.96682200979063,43.7597135970292,kplr2009170043915_ffi-cal.fits[43],meidian_41_43.fits,diff_imNo10_channel41_ccd43.fits,1134476,-0.4774268980651208,10.8538,22.347987687656364
839,39519.03,177.9088,13.508,0.0049,13.6558,962.863,0.529,295.98962279331516,40.34854231188855,kplr2009170043915_ffi-cal.fits[6],meidian_78_6.fits,diff_imNo10_channel78_ccd6.fits,665937,-0.6570662253972581,12.6507,12.511597151240595
416,16231.76,18.58386,14.4741,0.0012,925.7225,162.2713,0.511,285.20670164903896,41.9813239669527,kplr2009231194831_ffi-cal.fits[33],meidian_49_33.fits,diff_imNo30_channel49_ccd33.fits,513656,-2.3072737919547217,14.336,11.347083742108287
83,15776.54,11.82023,14.505,0.0008,328.1851,830.0725,0.29,288.1480409780076,50.575179784264684,kplr2009292020429_ffi-cal.fits[31],meidian_3_31.fits,diff_imNo39_channel3_ccd31.fits,1365403,-1.7261691265101389,14.2573,10.60225582688671


In [24]:
ascii.write(matchesSN,'/home/mj1e16/outTablesImproved/final/matchSNabove10.csv',format='csv',fast_writer=False,overwrite=True)

In [56]:
matchesSN

[]

In [45]:
df = matchesSN.to_pandas()

In [46]:
df2 = df[df['S/N']>= 100]

In [47]:
print(len(df2))

0


In [48]:
sn10Tab = matchesSN

In [49]:
sn100Tab = Table.from_pandas(df2)

In [50]:
ascii.write(matchesSN,'/home/mj1e16/outTablesImproved/final/10sn.csv',format='csv',fast_writer=False)

In [51]:
ascii.write(sn100Tab,'/home/mj1e16/outTablesImproved/final/100sn.csv',format='csv',fast_writer=False)

In [32]:
matchesSN

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index,magDiff,MAG_BEST_ORIG,S/N
int64,float64,float64,float64,float64,float64,float64,float64,float64,str34,str18,str32,int64,float64,float64,float64
0,2727.701,23.63741,16.4105,653.9053,25.5843,0.02,288.1118481431912,39.18350731406475,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379136,-0.006303316674529896,10.817,484.68927448514063
2,1927.37,21.8672,16.7876,1031.1854,24.3514,0.085,287.8050137546835,39.52618155323751,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379138,-0.006095044565130538,11.1577,414.32728900780035
3,2385.084,23.96147,16.5562,960.5677,23.126,0.12,287.86438045789714,39.463027137726364,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379139,-0.005330957156700023,10.7813,492.8311431798538
5,649.8285,10.05581,17.968,896.9971,34.9938,0.008,287.9025289569676,39.397941923071194,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379141,-0.029107763794769186,14.0242,110.10146942655714
7,2442.387,29.65171,16.5305,379.8951,50.3445,0.185,288.30496285987044,38.919117438701925,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379143,-0.004379558470541156,10.5425,550.2251173744397
26,735.3299,11.10855,17.8338,175.8217,983.0233,0.032,287.39072398438014,38.14359298103746,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379162,-0.00865991365777597,12.5839,214.70199079366958
27,721.2819,10.55474,17.8547,89.3047,982.0429,0.215,287.4618940251023,38.06613637752199,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379163,-0.00279190516320127,11.3788,374.4982787301537
29,413.3072,6.927397,18.4593,503.9937,933.1885,0.18,287.1809488700493,38.4713026614676,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379165,-0.0067198689791502664,12.9351,182.72838035916485
49,624.3866,10.3083,18.0114,808.2428,834.5024,0.138,287.0453176508683,38.80846647596356,kplr2009114174833_ffi-cal.fits[10],meidian_70_10.fits,diff_imNo11_channel70_ccd10.fits,379185,-0.004667797836333521,12.0925,269.4754858210636


In [48]:
fullMatches = matchesSN
df = fullMatches.to_pandas()
df = df[df['magDiff'] < -1]

In [49]:
print(len(df))
match25mag = Table.from_pandas(df)

851


In [50]:
ascii.write(match25mag,'/home/mj1e16/outTablesImproved/match1mag.csv',format='csv',fast_writer=False)

In [179]:
ascii.write(matchesSN,'/home/mj1e16/outTablesImproved/matchSNabove10.csv',format='csv',fast_writer=False)

In [180]:
print(len(matchesSN))

71198


In [ ]:
matches = []
for x in range(len(uni)):
    name = uni[x]
    print(name,x)
    diffname = unmatchedObjects['DIFF_NAME'][x]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['IMAGE_NAME'] == name]
    minitab = Table.from_pandas(mindf)
    indexing = range(len(minitab))
    minitab['NUMBER'] = indexing
                
    try:
        matchtab = ascii.read('/home/mj1e16/outTablesImproved/imagecsvredo/'+name+'.csv')
        indexing = range(len(matchtab))
        matchtab['NUMBER'] = indexing
        
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()

    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,max_sep=2.5*u.arcsec)
    onematchTab = Table.from_pandas(onematch)
    if len(onematch) != 0:
        if len(matches) == 0:
            onematchTab = Table.from_pandas(onematch)
            matches = onematchTab

        else: 
            #onematchTab = Table.from_pandas(onematch)
            #print(onematchTab)
            for x in range(len(onematchTab)):
                matches.add_row(onematchTab[x])
                #print(onematchTab)
                #matches  = vstack(matches,onematchTab)
                #print(matches)

In [156]:
df = matches.to_pandas()
df2 = df[df['magDiff'] <-5]
# print(len(df2))
# df2

In [157]:
matches5 = Table.from_pandas(df2)

In [77]:
# image2diff = {}
# image2median = {}
# for x in diffImages:
#     try:
#         results = findName(x)
#         image2diff[results[0]] = x
#         image2median[results[0]] = results[1]
#     except:
#         print(x)
    
# diff = [image2diff[x] for x in imname]
# median = [image2median[x] for x in imname]

# unmatchedObjects['diffImage'] = diff
# unmatchedObjects['medianImage'] = median

diff_imNo0_channel8_ccd36.fits


In [ ]:
##############################################  SUPERNOVAE WORK ##########################################################

# imges 
# ccds = [46,6,41]
# sneImages = ['kplr2012242195726_ffi-cal.fits[46]','kplr2011271191331_ffi-cal.fits[6]','kplr2012004204112_ffi-cal.fits[41]']

# goods = [41,94,97]
# for x in goods:
#     name = uniqueNameSort[x]
#     diffname = image2diff[name]
#     df = unmatchedObjects.to_pandas()
#     mindf = df[df['NAME'] == name]
#     minitab = Table.from_pandas(mindf)
#     try:
#         matchtab = ascii.read('/data/mj1e16/kepler/sexKeplerCal/'+name+'.csv')   
#     except:
#         matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
#     #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
#     bigdf = matchtab.to_pandas()
#     #meddf = mediantab.to_pandas()
#     onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
#     showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')
#     print(x)

In [19]:
# answer = [{'index': 45, 'timediff': 7.854872060001071, 'image': 'kplr2012242195726_ffi-cal.fits'},
#           {'index': 34, 'timediff': 13.539371960003336, 'image': 'kplr2011271191331_ffi-cal.fits'},
#           {'index': 37, 'timediff': 2.427519040000334, 'image': 'kplr2012004204112_ffi-cal.fits'}]
# ds9Options = []
# ccd = [45,1,44]
# for x in range(len(answer)):
#     imname = answer[x]['image']+'[{}]'.format(ccd[x])
#     subprocess.call(['ds9',imageDir+imname,'-regions','supernovae{}.reg'.format(x),'-scale','mode','zscale',
#                     diffDir+image2diff[imname],'-regions','supernovae{}.reg'.format(x),'-scale','mode','zscale',
#                     medianDir+image2median[imname],'-scale','mode','zscale','-frame','lock','image'])

In [67]:
# bigNames = matches['IMAGE_NAME']
# uniqueName = np.unique(bigNames)
# magdiff = matches['magDiff']
# magdiffsort = sorted(magdiff)
# uniqueNameSort = [x for _,x in sorted(zip(magdiffsort,uniqueName))]
# print(len(uniqueName))

14


In [ ]:
ascii.write(matches,'/home/mj1e16/outTablesImproved/matchplus3Magdiff.csv',format='csv',fast_writer=False)

In [158]:
ascii.write(matches5,'/home/mj1e16/outTablesImproved/matchplus5Magdiff.csv',format='csv',fast_writer=False)

In [60]:
#fullTab = ascii.read('/home/mj1e16/outTablesImproved/fullMatches.csv')

In [165]:
for x in range(len(matches5)):
    openFromTable(matches5,x)

https://ps1images.stsci.edu/rings.v3.skycell/2275/064/rings.v3.skycell.2275.064.stk.g.unconv.fits
(281.75827968509583, 46.712644169501274, 'kplr2009114204835_ffi-cal.fits[14]')
https://ps1images.stsci.edu/rings.v3.skycell/2210/090/rings.v3.skycell.2210.090.stk.g.unconv.fits
(283.1430488833758, 43.8178082207455, 'kplr2009114204835_ffi-cal.fits[2]')
https://ps1images.stsci.edu/rings.v3.skycell/2211/038/rings.v3.skycell.2211.038.stk.g.unconv.fits
(284.00090488071123, 41.26312629018617, 'kplr2009322233047_ffi-cal.fits[80]')
https://ps1images.stsci.edu/rings.v3.skycell/2276/092/rings.v3.skycell.2276.092.stk.g.unconv.fits
(288.3155839078486, 47.834189945327026, 'kplr2011053174401_ffi-cal.fits[24]')
https://ps1images.stsci.edu/rings.v3.skycell/2275/002/rings.v3.skycell.2275.002.stk.g.unconv.fits
(282.6474648663523, 44.08765699121511, 'kplr2011145152723_ffi-cal.fits[1]')
https://ps1images.stsci.edu/rings.v3.skycell/2275/096/rings.v3.skycell.2275.096.stk.g.unconv.fits
(280.4266698423564, 47.745

In [64]:
subprocess.call(['ds9', '/data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[57]', '-regions', '/home/mj1e16/outTablesImproved/testreg.reg', '-scale', 'mode', 'zscale', '/data/mj1e16/kepler/properDiff/diff_imNo6_channel25_ccd57.fits', '-regions', '/home/mj1e16/outTablesImproved/testreg.reg', '-scale', 'mode', 'zscale', '/data/mj1e16/kepler/medians/meidian_25_57.fits', '-regions', '/home/mj1e16/outTablesImproved/testreg.reg', '-scale', 'mode', 'zscale', '-frame', 'lock', 'image'])

0

In [203]:
goods = [41,94,97]
for x in goods:
    name = uniqueNameSort[x]
    diffname = image2diff[name]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['NAME'] == name]
    minitab = Table.from_pandas(mindf)
    try:
        matchtab = ascii.read('/data/mj1e16/kepler/sexKeplerCal/'+name+'.csv')   
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()
    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
    showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')
    print(x)

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in log10
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


41


IOError: [Errno 2] No such file or directory: '/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/kplr2009115131122_ffi-cal.fits[50].csv'